In [1]:
import cv2
import math
import numpy as np
from ultralytics import YOLO
from src.utils import calculate_distance
from src.utils import calculate_bounding_box_area
from src.utils import determine_side
from src.utils import plot_dynamic_chart
from src.utils import calculate_angle
import matplotlib.pyplot as plt

In [ ]:
# 对视频按照深蹲次数进行切割
# Load YOLOv8 pose estimation model
model = YOLO('yolov8l-pose.pt')

# Open the video
cap = cv2.VideoCapture('sample6.mp4')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 定义视频输出格式
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter('trimmed_output.mp4', fourcc, fps, (width, height))

# 变量来跟踪深蹲的状态
initial_shoulder_pos = None
squat_start = False
segment_count = 0
current_video_writer = None

threshold_down = 15  # 用于检测肩膀显著下移的阈值（单位：像素）
threshold_up = 10   # 用于检测肩膀回到初始位置的阈值
threshold_horizontal = 10

extra_frames = 10   # 深蹲结束后，延长的帧数
extra_frame_counter = 0

# 遍历视频的每一帧
frame_count = 0
results = model('sample6.mp4')  # 使用 YOLO 模型进行姿态检测

for result in results:
    frame_count += 1
    ret, frame = cap.read()
    if not ret:
        break

    # 获取关键点
    all_keypoints = result.keypoints.data  # 所有关键点

    max_area = 0
    largest_person = None

    # 找到检测到的最大目标
    for keypoints in all_keypoints:
        area = calculate_bounding_box_area(keypoints)
        if area > max_area:
            max_area = area
            largest_person = keypoints

    if largest_person is not None:
        # 获取左肩膀的坐标
        left_shoulder = largest_person[5][:2].cpu().numpy()
        left_hip = largest_person[11][:2].cpu().numpy()
        left_knee = largest_person[13][:2].cpu().numpy()
        left_ankle = largest_person[15][:2].cpu().numpy()

        right_shoulder = largest_person[6][:2].cpu().numpy()
        right_hip = largest_person[12][:2].cpu().numpy()
        right_knee = largest_person[14][:2].cpu().numpy()
        right_ankle = largest_person[16][:2].cpu().numpy()

        side, shoulder, hip, knee, ankle = determine_side(side, left_shoulder, left_hip, left_knee, left_ankle, right_shoulder, right_hip, right_knee, right_ankle)
        if initial_shoulder_pos is None:
            # 初始化肩膀的初始位置
            initial_shoulder_pos = shoulder
            continue

        # 计算肩膀的位移（Y 轴）
        shoulder_vertical_movement = shoulder[1] - initial_shoulder_pos[1]
        shoulder_horizontal_movement = abs(shoulder[0] - initial_shoulder_pos[0])
        # 检测深蹲开始：肩膀显著向下移动
        if shoulder_vertical_movement > threshold_down and not squat_start and shoulder_horizontal_movement<threshold_horizontal:
            squat_start = True
            extra_frame_counter = 0  # 重置延长帧计数器
            segment_count += 1
            # 创建一个新的视频文件来保存这一段
            current_video_writer = cv2.VideoWriter(f'squat_segment_{segment_count}.mp4', fourcc, fps, (width, height))
            print(f"Starting squat segment {segment_count}")

        # 检测深蹲结束：肩膀回到接近初始位置
        elif shoulder_vertical_movement < threshold_up and squat_start:
            if extra_frame_counter == 0:
                print(f"Ending squat segment {segment_count}, but continuing for {extra_frames} more frames.")
            extra_frame_counter += 1
            if extra_frame_counter >= extra_frames:
                squat_start = False
                # 关闭当前视频片段保存
                if current_video_writer is not None:
                    current_video_writer.release()
                    current_video_writer = None
                print(f"Completely ended squat segment {segment_count}")

        # 写入帧到当前视频段
        if squat_start or (extra_frame_counter > 0 and extra_frame_counter <= extra_frames):
            if current_video_writer is not None:
                current_video_writer.write(frame)

# 释放资源
cap.release()
if current_video_writer is not None:
    current_video_writer.release()

In [4]:
# 加载 YOLOv8 姿态估计模型
model = YOLO('yolov8l-pose.pt')
file_index = 1
file_name = 'squat_segment_{}.mp4'.format(file_index)
output_file = 'output_with_plots_{}.mp4'.format(file_index)

# 打开深蹲视频
cap = cv2.VideoCapture(file_name)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_width = frame_width + 500  # 增加500px的宽度用于图表
out = cv2.VideoWriter(output_file, fourcc, fps, (output_width, frame_height))

time_values = []
speed_values = []
knee_angles = [] 
hip_angles = []  
frame_number = 0
previous_position = None
side = None

results = model(file_name)

for result in results:
    ret, frame = cap.read()
    if not ret:
        break

    all_keypoints = result.keypoints.data  # 所有关键点

    max_area = 0
    largest_person = None

    for keypoints in all_keypoints:
        area = calculate_bounding_box_area(keypoints)
        if area > max_area:
            max_area = area
            largest_person = keypoints

    if largest_person is not None:
        left_shoulder = largest_person[5][:2].cpu().numpy()
        left_hip = largest_person[11][:2].cpu().numpy()
        left_knee = largest_person[13][:2].cpu().numpy()
        left_ankle = largest_person[15][:2].cpu().numpy()

        right_shoulder = largest_person[6][:2].cpu().numpy()
        right_hip = largest_person[12][:2].cpu().numpy()
        right_knee = largest_person[14][:2].cpu().numpy()
        right_ankle = largest_person[16][:2].cpu().numpy()

        side, shoulder, hip, knee, ankle = determine_side(side, left_shoulder, left_hip, left_knee, left_ankle, right_shoulder, right_hip, right_knee, right_ankle)

        if previous_position is None:
            previous_position = (shoulder + hip) / 2
            frame_number += 1
            continue

        knee_angle = calculate_angle(hip, knee, ankle)
        knee_angles.append(knee_angle)

        hip_angle = calculate_angle(shoulder, hip, knee)
        hip_angles.append(hip_angle)
        
        avg_position = (shoulder + hip) / 2
        vertical_movement = previous_position[1] - avg_position[1]
        frame_time = frame_number / fps
        speed = vertical_movement * fps
        time_values.append(frame_time)
        speed_values.append(speed)

        previous_position = avg_position

        distance1 = calculate_distance(hip, knee)
        distance2 = calculate_distance(knee, ankle)
        distance3 = calculate_distance(shoulder, hip)
        distance4 = distance1 + distance2 
        ratio1 = distance1 / distance2
        ratio2 = distance3 / distance4

        annotated_frame = result.plot()
        cv2.putText(annotated_frame, f"Femur Length: {distance1:.2f} px", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (238, 173, 14), 2)
        cv2.putText(annotated_frame, f"Tibia Length: {distance2:.2f} px", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (238, 173, 14), 2)
        cv2.putText(annotated_frame, f"Torso Length: {distance3:.2f} px", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (238, 173, 14), 2)
        cv2.putText(annotated_frame, f"Leg Length: {distance4:.2f} px", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (238, 173, 14), 2)
        cv2.putText(annotated_frame, f"Femur/Tibia: {ratio1:.2f}", (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (238, 173, 14), 2)
        cv2.putText(annotated_frame, f"Torso/Leg: {ratio2:.2f}", (50, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (238, 173, 14), 2)

        plot_image = plot_dynamic_chart(time_values, speed_values, knee_angles, hip_angles)
        plot_image_resized = cv2.resize(plot_image, (500, frame_height))

        combined_frame = np.hstack((annotated_frame, plot_image_resized))
        out.write(combined_frame)

    frame_number += 1

cap.release()
out.release()

print(f"Output video saved as {output_file}")




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/83) d:\software-engineer\TITAN TRACK\Software-Engineering-for-Data-Science\squat_segment_1.mp4: 640x384 1 person, 106.3ms
video 1/1 (frame 2/83) d:\software-engineer\TITAN TRACK\Software-Engineering-for-Data-Science\squat_segment_1.mp4: 640x384 1 person, 23.0ms
video 1/1 (frame 3/83) d:\software-engineer\TITAN TRACK\Software-Engineering-for-Data-Science\squat_segment_1.mp4: 640x384 1 person, 22.7ms
video 1/1 (frame 4/83) d:\software-en

In [15]:
import cv2
import numpy as np
from ultralytics import YOLO

# 加载 YOLOv8 姿态估计模型
model = YOLO('yolov8l-pose.pt')
file_index = 2
file_name = 'squat_segment_{}.mp4'.format(file_index)
output_file = 'output_with_plots_{}.mp4'.format(file_index)

# 打开深蹲视频
cap = cv2.VideoCapture(file_name)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_width = 2*frame_width + 500  # 增加1000px的宽度用于两个额外的部分（关节轨迹和图表）
out = cv2.VideoWriter(output_file, fourcc, fps, (output_width, frame_height))

# 初始化变量
time_values = []
speed_values = []
knee_angles = []
hip_angles = []
frame_number = 0
previous_position = None
side = None

# 运行YOLO模型进行姿态估计
results = model(file_name)

# 循环处理每一帧
for result in results:
    ret, frame = cap.read()
    if not ret:
        break

    # 获取所有关键点
    all_keypoints = result.keypoints.data
    max_area = 0
    largest_person = None

    # 查找最大面积的人物
    for keypoints in all_keypoints:
        area = calculate_bounding_box_area(keypoints)
        if area > max_area:
            max_area = area
            largest_person = keypoints

    if largest_person is not None:
        # 获取左侧和右侧的关键点坐标
        left_shoulder = largest_person[5][:2].cpu().numpy()
        left_hip = largest_person[11][:2].cpu().numpy()
        left_knee = largest_person[13][:2].cpu().numpy()
        left_ankle = largest_person[15][:2].cpu().numpy()

        right_shoulder = largest_person[6][:2].cpu().numpy()
        right_hip = largest_person[12][:2].cpu().numpy()
        right_knee = largest_person[14][:2].cpu().numpy()
        right_ankle = largest_person[16][:2].cpu().numpy()

        # 确定侧面（左侧或右侧）并更新相关关键点
        side, shoulder, hip, knee, ankle = determine_side(side, left_shoulder, left_hip, left_knee, left_ankle, right_shoulder, right_hip, right_knee, right_ankle)

        # 如果是第一帧，初始化previous_position
        if previous_position is None:
            previous_position = (shoulder + hip) / 2
            frame_number += 1
            continue

        # 计算膝盖角度和髋部角度
        knee_angle = calculate_angle(hip, knee, ankle)
        knee_angles.append(knee_angle)

        hip_angle = calculate_angle(shoulder, hip, knee)
        hip_angles.append(hip_angle)

        # 计算速度
        avg_position = (shoulder + hip) / 2
        vertical_movement = previous_position[1] - avg_position[1]
        frame_time = frame_number / fps
        speed = vertical_movement * fps
        time_values.append(frame_time)
        speed_values.append(speed)

        previous_position = avg_position

        # 计算关键点之间的距离
        distance1 = calculate_distance(hip, knee)
        distance2 = calculate_distance(knee, ankle)
        distance3 = calculate_distance(shoulder, hip)
        distance4 = distance1 + distance2
        ratio1 = distance1 / distance2
        ratio2 = distance3 / distance4

        # 在原始视频帧上添加注释
        annotated_frame = result.plot()
        cv2.putText(annotated_frame, f"Femur Length: {distance1:.2f} px", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (238, 173, 14), 2)
        cv2.putText(annotated_frame, f"Tibia Length: {distance2:.2f} px", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (238, 173, 14), 2)
        cv2.putText(annotated_frame, f"Torso Length: {distance3:.2f} px", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (238, 173, 14), 2)
        cv2.putText(annotated_frame, f"Leg Length: {distance4:.2f} px", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (238, 173, 14), 2)
        cv2.putText(annotated_frame, f"Femur/Tibia: {ratio1:.2f}", (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (238, 173, 14), 2)
        cv2.putText(annotated_frame, f"Torso/Leg: {ratio2:.2f}", (50, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (238, 173, 14), 2)

        # 创建用于绘制关键点运动的帧
        keypoints_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)  # 空白帧，用于绘制关键点
        cv2.circle(keypoints_frame, tuple(shoulder.astype(int)), 5, (0, 255, 0), -1)  # 肩膀
        cv2.circle(keypoints_frame, tuple(hip.astype(int)), 5, (0, 255, 0), -1)       # 髋部
        cv2.circle(keypoints_frame, tuple(knee.astype(int)), 5, (0, 255, 0), -1)      # 膝盖
        cv2.circle(keypoints_frame, tuple(ankle.astype(int)), 5, (0, 255, 0), -1)     # 脚踝

        # 绘制关节之间的连线
        cv2.line(keypoints_frame, tuple(shoulder.astype(int)), tuple(hip.astype(int)), (255, 0, 0), 2)
        cv2.line(keypoints_frame, tuple(hip.astype(int)), tuple(knee.astype(int)), (255, 0, 0), 2)
        cv2.line(keypoints_frame, tuple(knee.astype(int)), tuple(ankle.astype(int)), (255, 0, 0), 2)

        # 绘制动态图表
        plot_image = plot_dynamic_chart(time_values, speed_values, knee_angles, hip_angles)
        plot_image_resized = cv2.resize(plot_image, (500, frame_height))  # 调整图表大小

        # 将原始视频帧、keypoints_frame（关键点帧）和图表帧拼接在一起
        combined_frame = np.hstack((keypoints_frame, annotated_frame, plot_image_resized))
        out.write(combined_frame)

    frame_number += 1

cap.release()
out.release()

print(f"Output video saved as {output_file}")




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/92) d:\software-engineer\TITAN TRACK\Software-Engineering-for-Data-Science\squat_segment_2.mp4: 640x384 1 person, 49.9ms
video 1/1 (frame 2/92) d:\software-engineer\TITAN TRACK\Software-Engineering-for-Data-Science\squat_segment_2.mp4: 640x384 1 person, 33.0ms
video 1/1 (frame 3/92) d:\software-engineer\TITAN TRACK\Software-Engineering-for-Data-Science\squat_segment_2.mp4: 640x384 1 person, 34.4ms
video 1/1 (frame 4/92) d:\software-eng